# GeoDN Course 2: Fundamentals of Geospatial Data and Modeling - Part 1 Geospatial Data Discovery for Climate Risk #
> Copyright (c) 2024 International Business Machines Corporation

> This software is released under the MIT License.
> https://opensource.org/licenses/MIT

# Session 2 - Flood model and real flood extent with buildings

Once we have the information about floods and extreme rainfall events, we want to understand the impact this has on buildings or general infrastructure. For this, we need to combine the flood outlines with the buildings for our area of interest. We also compare that to the flood model output and see how much the model output matches the real events. 

In this Notebook you will learn to: 

(1) Load our shapefile of interest  
(2) Load our flood information (in this case a shapefile with flood extends - true data)   
(3) Get buildings from OpenStreetMaps   
(4) Calculate interesection between buildings and floods   
(5) Map buildings and flood extents   
(6) Create overview of buildings affected   
(7) Compare the real flood extents to the predicted flood extents from the model    

### Prepare

Load import modules



In [ ]:
from pathlib import Path
import seaborn as sns
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
import plotly.express as px
import plotly.graph_objects as go
import folium
%matplotlib inline

<font color='white'>********************************************************************************************************************************************************************</font>

# 1) Load shapefiles for Hull

In our case this is still the Hull shapefile. This is a shapefile that contains the outline of Hull City. We use this so that we have the coordinates and outlines as a geospatial file. We can then use that later to pull data for that specific outline of Hull. 
We will then plot it on a nice map with `folium` where you can zoom in and out. For the map, we specify the center location with latitude and longitude coordinates and specify a certain map style, in this case we use `tiles="CartoDB Positron"` but you can choose others, see here https://python-visualization.github.io/folium/latest/user_guide/raster_layers/tiles.html

In [ ]:
#Load file
shpFilePath='shapefiles/cutline.shp'
hull = gpd.read_file(shpFilePath)

#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
hull.explore(m=map,color='blue')



<font color='white'>********************************************************************************************************************************************************************</font>

# 2) Load flood extents

We load the flood extents and change the coordinate system to match `ESPG:4326` from the shapefile for Hull. These flood events are the real flood outlines from the 2007 floods in Hull. They only contain information about the outline of the water, but not how deep the water was. We also clip the flood file to the same shape as our shapefile from Hull was above, so that we save a bit on processing. 

In [ ]:
shpFilePath_flood='shapefiles/hull-flood-outlines.shp'
flood = gpd.read_file(shpFilePath_flood)
# Make sure coordinate system is set to world grid EPSG:4326
flood = flood.to_crs(4326)
#Clip flood extents to Hull only 
flood = gpd.clip(flood, hull)

#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
flood.explore(m=map,color='red')

Let's do a quick tidy up to make processing easier for later:

In [ ]:
#Tidy up dataframe by removing unnecessary columns
flood
flood = flood[['name','start_date','flood_caus', 'fluvial_f', 'coastal_f', 'tidal_f', 'hfm_status','geometry']]


The flood extents contain `MULTIPOLYGONS` as well as single `POLYGONS`. Multipolygons are nested/stitched together Polygons, as the name suggests. The `intersect` function in `geopandas` is not able to compute the intercept when there is a mix of multi and single polygons, so we use the `explode` function to convert the multipolygons into single polygons, each will be added as a new row with the same metainfo. 

In [ ]:
#Convert multipolygons in single polygons for later intercept calculation. 
flood_exploded = flood.explode(ignore_index=True)
flood_exploded

<font color='white'>********************************************************************************************************************************************************************</font>

# 3) Get buildings from OpenStreetMaps

We use OpenStreetmaps to get all buildings outlines for our area of interest. OpenStreetMaps has a tag system where everything on a map is classified into a building, road, land etc. Each tag has then a specific key which gives more details. For example a shape on OSM is tagged as a building and the specific type (key) is "apartment". 

We extracted all the buildings from OpenStreetMaps, see here all available types: https://wiki.openstreetmap.org/wiki/Map_features
    

We set up the function to pull the data from our Hull Polygon Shapefile as loaded above. 

In [ ]:
# Set all possible building types
building_types = [
    "apartments", "barracks", "bungalow", "cabin", "detached",
    "dormitory", "farm", "ger", "hotel", "house", "houseboat",
    "residential", "semidetached_house", "static_caravan", "commercial", "industrial",
    "kiosk", "office", "retail", "supermarket", "warehouse", "cathedral", "chapel",
    "church", "monastery", "mosque", "presbytery", "religious", "shrine", "synagogue",
    "temple", "bakehouse", "civic", "college", "fire_station", "government", "hospital",
    "kindergarten", "public", "school", "toilets", "train_station", "transportation",
    "university", "barn", "conservatory", "cowshed", "farm_auxiliary", "greenhouse",
    "slurry_tank", "stable", "sty", "grandstand", "pavilion", "riding_hall", "sports_hall",
    "stadium", "hangar", "hut", "shed", "carport", "garage", "garages", "parking", "digester",
    "service", "transformer_tower", "water_tower", "beach_hut", "bunker", "bridge", "castle",
    "construction", "container", "gatehouse", "military", "roof", "ruins", "tent", "tree_house"
]


Then we set up the function to retrieve the buildings from OSM. 

In [ ]:
def get_buildings(polygon, building_type):

    '''
    This function uses the osmnx package to get data from 
    Open Street Maps. 
    Tags are used to define what data you request. 
    https://wiki.openstreetmap.org/wiki/Tags
    We specified the different building types, see here the full list: 
    https://wiki.openstreetmap.org/wiki/Key:building
    '''
    
    # We are interested in buildings
    tags = {"building": building_type}
    
    # Acquire buildings' Geodataframe for polygon
    gdf = ox.features_from_polygon(polygon, tags) 
    
    # Add the tag to it
    gdf["building_type"] = building_type
    
    return gdf

Now run this function for a place of choice and set a path where it is saved as a shapefile, so we don't have to re-run this again. 


In [ ]:
# Set the data path

data_path = Path("buildings.shp")

polygon = hull.loc[0, 'geometry'] #we use the polygon for hull from the geodataframe
gdfs = list()
for building_type in building_types:
    print(f"Acquiring {building_type}....")
    try:
        gdfs.append(get_buildings(polygon, building_type))
    except:
        print(f"{building_type} not in this area.")

# Concatenate all geodataframes
gdf = pd.concat(gdfs).reset_index()
gdf = gdf[["building", "geometry"]]
gdf = gdf.rename(columns={"building": "btype"})
gdf = gdf[gdf["geometry"].type != "Point"]
    
# Save
gdf.to_file(data_path)

What is returned is a geodataframe, containing the building type (`btype`) and Polygon information about this building. The Polygon information let's us plot the buildings on a map and we use those Polygon outlines to match/intersect with the flood outlines later on. 

In [ ]:
gdf

Plot buildings on a map to make sure we have the right area. As you can see on the area, not all buildiings are pulled with the OpenStreetMaps Package. Some areas work better than others, it depends how well an area has been manually annotated. For this exercise it doesn't really matter but needs to be taken into account if you want to do more accurate analysis. 

In [ ]:
#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
gdf.explore(m=map,color='green')

<font color='white'>********************************************************************************************************************************************************************</font>

# 4) Calculate interesection between buildings and floods

Now we want to show the intersect between the flood outlines and the building outlines. We check the dataframe for the flood extents and the buildings to make sure they are all ready to go. 

In [ ]:
gdf

In [ ]:
flood_exploded

Now run function to calculate the intersect between the floods and buildings. The output is a dataframe of affected buildings. 

In [ ]:
build_affected = gpd.overlay(gdf, flood_exploded, how='intersection')
build_affected




<font color='white'>********************************************************************************************************************************************************************</font>

# 5) Map buildings and flood extents

Now let's actually map the affected buildings after we run the interesection between buildings and flood extents. Note that previously we mapped all flood and building outlines and now we're just plotting the buildings that are affected by the flood event itself. 

In [ ]:
#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
build_affected.explore(m=map,color='red')

We can also use the plot to look at the flood extents on the map and explore it a little further in the next section. For this, we plot the flood extents on the same map from the previous cell by simply adding it to the same map object. 

In [ ]:
# Add the GeoDataFrame to the map.
flood_exploded.explore(m=map,color='blue')

We also add all buildings again together with the flooded buildings and the flood extents, which shows nicely how everything overlaps. We re-order everything so that the flood extents are the bottom layer. Zoom in to explore

In [ ]:
#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
flood_exploded.explore(m=map,color='blue')

# Add the GeoDataFrame to the map.
gdf.explore(m=map,color='green')

# Add the GeoDataFrame to the map.
build_affected.explore(m=map,color='red')

<font color='white'>********************************************************************************************************************************************************************</font>

# 6) Create overview of buildings affected

We want to understand how many buildings are affected by the flood in total, so we create some stats and plots based on the dataframe of the intersection. 

In [ ]:
build_affected

In [ ]:
print("Total frequency of affected buildings")
print(build_affected.btype.value_counts())

print("Total flood events relevant")
print(build_affected.name.value_counts())

In [ ]:
#Do a frequency count on flood depth and building type
df = build_affected.copy()
df = df.groupby(['btype']).count().reset_index()
# Delete other columns: 
df = df[['btype','name']]
# Rename columns
df.columns=['btype','frequency']


#plot
sns.set(rc={'figure.figsize':(15,10)})
ax = sns.barplot(data=df, x="btype", y="frequency", hue='btype',palette = "hls",log=True, width=1,dodge = False)
ax.set(xlabel='Building Type', ylabel='frequency',title='assets affected by floods')
plt.legend([],[], frameon=False)
plt.xticks(rotation=90)



<font color='white'>********************************************************************************************************************************************************************</font>

# 7) Compare real floods to model output floods


EA surface water flood risk maps - these are risk maps created by the environment agency for different storm scenarios. In this case, we use the scenario of an extreme storm, occurring once in 1000 years (called 1in1000). We can then see if this extreme storm scenario matches with the actual flood event in Hull as seen above.    

This step below takes a few minutes as the shapefile is quite large. 

In [ ]:
#Load shapefile
shpFilePath_flood='shapefiles/hull_ea_flood_extent.shp'
flood_model = gpd.read_file(shpFilePath_flood)
flood_model.plot()



In [ ]:
flood_model

Now we can compare the flood model output to the real floods and see how they match. We plot again on our familiar map: 

In [ ]:
#Specify map object
map = folium.Map(location=[53.76,-0.35], tiles="CartoDB Positron", zoom_start=11)

# Add the GeoDataFrame to the map.
flood_exploded.explore(m=map,color='blue')

# Add the GeoDataFrame to the map.
flood_model.explore(m=map,color='red')


From this we can see that the actual flood was quite severe in some areas compared to what the Environment Agencies considered an extreme 1 in 1000 year storm, however the storm scenario was a lot more wide spread. We want to explore in the next section what the flood depth does in terms of building damage and what that would cost. 

# Explore more

Try running the OpenStreetMap data acquisition for a different location. You don't need to have a shapefile of a location, you can also search by place name (or a bounding box). The function above changes to this then: 

### From place: 
```
# We create a function to call all buildings for a place, in this case city
def get_buildings(place, building_type):
    
    # We are interested in buildings
    tags = {"building": building_type}
    
    # Acquire buildings' Geodataframe
    gdf = ox.features_from_place(place, tags) 
    
    # Add the tag to it
    gdf["building_type"] = building_type
    
    return gdf

# Set the data path

data_path = Path("buildings.shp")

place = "Berlin"
gdfs = list()
for building_type in building_types:
    print(f"Acquiring {building_type}....")
    try:
        gdfs.append(get_buildings(place, building_type))
    except:
        print(f"{building_type} not in this area.")

# Concatenate all geodataframes
gdf = pd.concat(gdfs).reset_index()
gdf = gdf[["building", "geometry"]]
gdf = gdf.rename(columns={"building": "btype"})
gdf = gdf[gdf["geometry"].type != "Point"]
    
# Save
gdf.to_file(data_path)
```

Or from bounding box, change the function to `features_from_bbox(north, south, east, west, tags)`

You can also run the intersection between the flood model and the actual flood to see how they compare in the overlapping areas and plot them on a map. 

# What's next

Next, we can move on to the next Notebook, where we can calculate the impact on flood depth vs building fragility. We will be using the Environment Agency Storm scenario for this, as this will have water depth information and not just the outlines of the floods. 